In [1]:
import requests, uuid, json
from time import sleep
from tqdm import tqdm

# Add your subscription key and endpoint
subscription_key = "1d79a0c2bf56409f8f63f51dc388dbe2"
endpoint = "https://api.cognitive.microsofttranslator.com"

# Add your location, also known as region. The default is global.
# This is required if using a Cognitive Services resource.
location = "eastus"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'en',
    'to': 'vi',
}
constructed_url = endpoint + path

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

In [2]:
size_text = 0
def by_microsoft(translation):
    global size_text
    size_text += len(translation)
    body = [{
        'text': translation, 
    }]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    response = response[0]['translations'][0]['text']
    translated = []
    eng = translation.split('.')
    vi = response.split('.')
    for i in range(len(eng)):
        translated.append('\n\n\n' + eng[i])
        if i < len(vi):
            translated.append('\n\n' + vi[i])
    return '.'.join(translated)

In [3]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import urllib.request
import shutil

import json
import numpy as np
import pandas as pd
import re
import os
import retry
import datetime

driver = webdriver.Chrome()
driver.get('https://www.wired.com/')

def xpath_send_key(xpath_, string_):
    driver.find_element(By.XPATH, xpath_).send_keys(string_)

def xpath_click(xpath_):
    driver.find_element(By.XPATH, xpath_).click()

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
articles = set()
for s in soup.find_all('a', {'data-recirc-pattern': 'summary-item'}):
    if 'https' not in s['href']:
        articles.add(s['href'])
articles

{'/1994/09/moon-land/',
 '/2009/11/ff-vanish2/',
 '/gallery/best-electric-scooters',
 '/gallery/best-laptops/',
 '/gallery/best-monitors',
 '/gallery/best-office-chairs/',
 '/gallery/lawn-and-outdoor-games/',
 '/review/audio-technica-ath-m20xbt/',
 '/review/fluidfreeride-fluid-mosquito/',
 '/story/age-of-social-credit/',
 '/story/best-disposable-face-masks-n95-kn95-kf94-surgical',
 '/story/best-rapid-at-home-covid-19-test-kits/',
 '/story/big-business-burying-carbon-dioxide-capture-storage/',
 '/story/collective-intelligence-democracy/',
 '/story/david-wengrow-dawn-of-everything/',
 '/story/google-tv-home-workouts/',
 '/story/how-clean-is-clean-hydrogen/',
 '/story/how-to-create-secure-folder-phone',
 '/story/inside-google-three-years-misery-happiest-company-tech/',
 '/story/john-deere-tractor-jailbreak-defcon-2022/',
 '/story/karl-friston-free-energy-principle-artificial-intelligence/',
 '/story/media-climate-change-film/',
 '/story/meet-the-lobbyist-next-door/',
 '/story/particle-phy

In [5]:
def scrap_article():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    title = soup.find('main').find('h1').text

    forbidden_char = ['<', '>', ':', '"', "'", '/', '\\', '|', '?', '*']

    file_name = title + '.txt'
    for c in forbidden_char:
        file_name = file_name.replace(c, '_')

    if file_name in os.listdir('articles/'):
        return

    need_translatation = ""
    for s in soup.find_all('div', {'class':'body__inner-container'}):
        for paragraph in s.find_all('p'):
            need_translatation += paragraph.text
    try:
        translated = by_microsoft(need_translatation)
    except Exception as e:
        print(e)
        return

    f = open('articles/' + file_name, 'w', encoding='utf-8')
    f.write('# ' + title + '\n')
    f.write(translated)
    f.close()

In [6]:
based_url = 'https://www.wired.com'
for article in tqdm(articles):
    print(article)
    driver.get(based_url + article)
    if 'I Accept' in str(BeautifulSoup(driver.page_source, 'html.parser')):
        xpath_click('//*[@id="onetrust-accept-btn-handler"]')
    scrap_article()
size_text

  0%|          | 0/32 [00:00<?, ?it/s]

/story/collective-intelligence-democracy/


  3%|▎         | 1/32 [00:08<04:30,  8.71s/it]

Không có n
/story/the-loneliness-of-the-junior-college-esports-coach/


  6%|▋         | 2/32 [00:24<06:29, 12.99s/it]

...


/story/best-rapid-at-home-covid-19-test-kits/


 69%|██████▉   | 22/32 [04:04<01:41, 10.17s/it]

Đại dịch v
/story/inside-google-three-years-misery-happiest-company-tech/


 69%|██████▉   | 22/32 [04:12<01:54, 11.50s/it]


KeyError: 0